In [44]:
import pandas as pd
from tsl.data.spatiotemporal_dataset import SpatioTemporalDataset
from tsl.ops.connectivity import SparseTensor
import torch
import numpy as np
from torch_geometric.utils import to_undirected
from pathlib import Path
path = "/home/jaydenfassett/amlproject/data/county_data"

In [45]:
df = pd.read_parquet(path)
graph = pd.read_csv("/home/jaydenfassett/amlproject/data/geographic/graph.csv")


In [52]:

df = df.reset_index()
# mean 0, var 1 all feature columns

for col in df.columns.drop(labels=["level", "county", "datetime"]):
    print("Normalizing",col)
    assert col != "index", "Re-run notebook if index is in cols"
    df[col] = (df[col] - df[col].mean()) / df[col].std()

grouped = df.groupby(['level', 'county']).first().reset_index()
uniques = grouped.columns.drop(labels=["level","county"])
times = grouped['level'].unique()
nodes = grouped['county'].unique()

fips_idx = {name: i for i, name in enumerate(nodes)} # FIPS CODE / local index map.

T, N, F = len(times), len(nodes), len(uniques) - 1
features = grouped[[i for i in uniques]].values

datetimes = features[:,0].reshape(T, N)
tensor_3d = features[:,1:].reshape(T, N, F).astype(np.float32)

# t, n, f = tensor_3d


Normalizing index


AssertionError: Re-run notebook if index is in cols

In [47]:
for col in ['src','dest']:
    graph[col] = graph[col].apply(lambda x: fips_idx[x])

In [48]:
x = torch.from_numpy(tensor_3d)

E = torch.from_numpy(graph[['src','dest']].values).T
# E = tuple(graph[['src','dest']].values.T)

E_w = torch.from_numpy(graph["total_voltage"].values)

E_w = (E_w - E_w.mean()) / E_w.std()


In [50]:
dataset = SpatioTemporalDataset(
    tensor_3d,                    
    mask=None,                    
    connectivity=(E,E_w),    # Takes edges & weights    
    horizon=1,                    
    window=3,                     
    stride=1,                     
)



In [51]:
savepath = Path("/home/jaydenfassett/amlproject/data/tsldataset")
savepath.mkdir(exist_ok=True,parents=True)

dataset.save(savepath / "dataset.tsl")